# Open-R1-Distill

- [huggingface/open-r1][2]
- [open-r1/Mixture-of-Thoughts][1]

[1]: https://huggingface.co/datasets/open-r1/Mixture-of-Thoughts
[2]: https://github.com/huggingface/open-r1

## 環境構築

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
# 環境変数の設定

import os

os.environ["VLLM_USE_V1"] = "0"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["LOCAL_RANK"] = "0"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12345"

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    if not os.path.exists("/content/open-r1"):
        %git clone https://github.com/huggingface/open-r1.git
    %cd /content/open-r1
    %pip install -e ".[dev]" --no-deps
except ImportError:
    IN_COLAB = False
    # !apt update && apt install git-lfs -y
    if not os.path.exists("/workspaces/open-r1-distill/open-r1"):
        %git clone https://github.com/huggingface/open-r1.git
    %cd /workspaces/open-r1-distill/open-r1
    %pip install -e ".[dev]" --no-deps

In [ ]:
# 1) PyTorchとTransformersのインストール
%pip install torch==2.6.0 transformers==4.52.3

# 2) vLLMのインストール
%pip install vllm==0.8.5.post1

# 3) Flash Attentionのインストール
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
%pip install "https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl" --no-build-isolation

# 4) その他の必要なパッケージのインストール
%pip install \
    accelerate==1.4.0 \
    beautifulsoup4 \
    "async-lru" \
    bitsandbytes \
    "distilabel[vllm]" \
    deepspeed==0.16.8 \
    hf_transfer \
    langdetect \
    latex2sympy2_extended \
    liger-kernel \
    "trl[vllm]==0.18.0" \
    math-verify==0.5.2 \
    wandb

In [ ]:
from datasets import load_dataset
from distilabel.models import vLLM
from distilabel.pipeline import Pipeline
from distilabel.steps.tasks import TextGeneration
import gc
import torch

## データセット構築

### データソースの読み込み

In [ ]:
# 72400件の数学データセット
# https://huggingface.co/datasets/AI-MO/NuminaMath-TIR

# テスト用に少しだけ取得
dataset = load_dataset(
    "AI-MO/NuminaMath-TIR",
    split="train",
).select(range(10))

logger.info(f"{len(dataset)=} {dataset[0].keys()=}")

In [ ]:
# problemのサンプルを表示
logger.info(f"Problem:\n{dataset[0]['problem']}")

# (3/5x - 2/y)^8の展開式におけるx^2y^6の係数を分数で求めよ

In [ ]:
# solutionのサンプルを表示
logger.info(f"Solution:\n{dataset[0]['solution']}")

In [ ]:
# messagesのサンプルを表示
logger.info(f'messages\n{dataset[0]["messages"]}')

### 生成パイプライン作成

In [ ]:
# ステップバイステップで推論し、最終回答を\boxed{}で囲ってください
prompt_template = """\
You will be given a problem. Please reason step by step, and put your final answer within \boxed{}:
{{ instruction }}"""

In [ ]:
# 蒸留パイプラインを作成

# 蒸留元のモデル
# https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

with Pipeline(
    name="distill-qwen-1.5b-r1",
    description="A pipeline to generate data from a distilled r1 model",
) as pipeline:

    # distilabelのvLLMを初期化
    # https://distilabel.argilla.io/dev/components-gallery/llms/vllm/?h=vllm
    llm = vLLM(
        model=model_id,
        tokenizer=model_id,
        extra_kwargs={
            "tensor_parallel_size": 1,
            "max_model_len": 1024, # 8192
        },
        generation_kwargs={
            "temperature": 0.6,
            "max_new_tokens": 1024, # 8192
        },
    )

    prompt_column = "problem"

    text_generation = TextGeneration(
        llm=llm, 
        template=prompt_template,
        num_generations=4, # 1つのデータに対して4つ生成
        input_mappings={"instruction": prompt_column} if prompt_column is not None else {}
    )

### 生成

In [ ]:
distiset = pipeline.run(dataset=dataset)

In [ ]:
# 10 * 4 = 40件データが生成される
logger.info(f"{distiset=}")

In [ ]:
# 蒸留元のモデル名
print(distiset["default"]["train"][0]["model_name"])

In [ ]:
# 生成プロンプト
print(distiset["default"]["train"][0]["distilabel_metadata"])

In [ ]:
# 生成結果
print(distiset["default"]["train"][0]["generation"])

## SFT

```sh
python src/open_r1/sft.py \
    --config recipes/OpenR1-Distill-7B/sft/config_distill.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none
```

In [ ]:
from dataclasses import dataclass, field
from datasets import DatasetDict, concatenate_datasets
from datasets import load_dataset
from functools import partial, update_wrapper
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse, verify
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer
from transformers import set_seed
from transformers import TrainerCallback
from transformers.trainer_utils import get_last_checkpoint
from transformers.training_args import TrainingArguments
from trl import GRPOTrainer, ModelConfig, TrlParser, get_peft_config
from trl import ModelConfig, get_kbit_device_map, get_quantization_config
from trl import ModelConfig, SFTTrainer, TrlParser, get_peft_config, setup_chat_format
from typing import Any, Callable, Dict, Literal, List, Optional
import asyncio
import datasets
import json
import math
import os
import re
import subprocess
import sys
import transformers
import trl

logger.info(f"transformers: {transformers.__version__}") # 4.52.3
logger.info(f"torch: {torch.__version__}") # 2.6.0+cu124
logger.info(f"trl: {trl.__version__}") # 0.18.0

### 設定クラス

In [ ]:
@dataclass
class ScriptArguments(trl.ScriptArguments):
    """
    Extended version of ScriptArguments with support for dataset mixtures.

    Args:
        dataset_mixture (`dict[str, Any]` or `None`, *optional*, defaults to `None`):
            Configuration for creating dataset mixtures with advanced options.
            Format:
              dataset_mixture:
                datasets:
                  - id: dataset_id1
                    config: config_name
                    columns:
                      - col1
                      - col2
                    weight: 0.5
                  - id: dataset_id2
                    config: config_name
                    columns:
                      - col1
                      - col2
                    weight: 0.5
                seed: 42
                test_split_size: 0.1
    """

    # Override the dataset_name to make it optional
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "Dataset name. Can be omitted if using dataset_mixture."}
    )
    dataset_mixture: Optional[dict[str, Any]] = field(
        default=None,
        metadata={"help": "Configuration for creating dataset mixtures with advanced options like shuffling."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.dataset_mixture is None:
            raise ValueError("Either `dataset_name` or `dataset_mixture` must be provided")

        if self.dataset_mixture is not None:
            if not isinstance(self.dataset_mixture, dict) or "datasets" not in self.dataset_mixture:
                raise ValueError(
                    "dataset_mixture must be a dictionary with a 'datasets' key. "
                    "Expected format: {'datasets': [...], 'seed': int}"
                )

            datasets_list = []
            datasets_data = self.dataset_mixture.get("datasets", [])

            if isinstance(datasets_data, list):
                for dataset_config in datasets_data:
                    datasets_list.append(
                        DatasetConfig(
                            id=dataset_config.get("id"),
                            config=dataset_config.get("config"),
                            split=dataset_config.get("split", "train"),
                            columns=dataset_config.get("columns"),
                            weight=dataset_config.get("weight", 1.0),
                        )
                    )
            else:
                raise ValueError("'datasets' must be a list of dataset configurations")

            self.dataset_mixture = DatasetMixtureConfig(
                datasets=datasets_list,
                seed=self.dataset_mixture.get("seed", 0),
                test_split_size=self.dataset_mixture.get("test_split_size", None),
            )

            # Check that column names are consistent across all dataset configs
            columns_sets = [set(dataset.columns) for dataset in datasets_list if dataset.columns is not None]
            if columns_sets:
                first_columns = columns_sets[0]
                if not all(columns == first_columns for columns in columns_sets):
                    raise ValueError(
                        "Column names must be consistent across all dataset configurations in a mixture. "
                        f"Found different column sets: {[list(cols) for cols in columns_sets]}"
                    )

In [ ]:
@dataclass
class SFTConfig(trl.SFTConfig):
    """
    args for callbacks, benchmarks etc
    """

    benchmarks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "The benchmarks to run after training."},
    )
    callbacks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "The callbacks to run during training."},
    )
    chat_template: Optional[str] = field(default=None, metadata={"help": "The chat template to use."})
    system_prompt: Optional[str] = field(
        default=None,
        metadata={"help": "The optional system prompt to use for benchmarking."},
    )
    hub_model_revision: Optional[str] = field(
        default="main",
        metadata={"help": "The Hub model branch to push the model to."},
    )
    overwrite_hub_revision: bool = field(default=False, metadata={"help": "Whether to overwrite the Hub revision."})
    push_to_hub_revision: bool = field(default=False, metadata={"help": "Whether to push to a Hub revision/branch."})
    wandb_entity: Optional[str] = field(
        default=None,
        metadata={"help": ("The entity to store runs under.")},
    )
    wandb_project: Optional[str] = field(
        default=None,
        metadata={"help": ("The project to store runs under.")},
    )
    wandb_run_group: Optional[str] = field(
        default=None,
        metadata={"help": ("The group to store runs under.")},
    )

### ユーティリティ関数

In [ ]:
def get_dataset(args: ScriptArguments) -> DatasetDict:
    """Load a dataset or a mixture of datasets based on the configuration.

    Args:
        args (ScriptArguments): Script arguments containing dataset configuration.

    Returns:
        DatasetDict: The loaded datasets.
    """
    if args.dataset_name and not args.dataset_mixture:
        logger.info(f"Loading dataset: {args.dataset_name}")
        return datasets.load_dataset(args.dataset_name, args.dataset_config)
    elif args.dataset_mixture:
        logger.info(f"Creating dataset mixture with {len(args.dataset_mixture.datasets)} datasets")
        seed = args.dataset_mixture.seed
        datasets_list = []

        for dataset_config in args.dataset_mixture.datasets:
            logger.info(f"Loading dataset for mixture: {dataset_config.id} (config: {dataset_config.config})")
            ds = datasets.load_dataset(
                dataset_config.id,
                dataset_config.config,
                split=dataset_config.split,
            )
            if dataset_config.columns is not None:
                ds = ds.select_columns(dataset_config.columns)
            if dataset_config.weight is not None:
                ds = ds.shuffle(seed=seed).select(range(int(len(ds) * dataset_config.weight)))
                logger.info(
                    f"Subsampled dataset '{dataset_config.id}' (config: {dataset_config.config}) with weight={dataset_config.weight} to {len(ds)} examples"
                )

            datasets_list.append(ds)

        if datasets_list:
            combined_dataset = concatenate_datasets(datasets_list)
            combined_dataset = combined_dataset.shuffle(seed=seed)
            logger.info(f"Created dataset mixture with {len(combined_dataset)} examples")

            if args.dataset_mixture.test_split_size is not None:
                combined_dataset = combined_dataset.train_test_split(
                    test_size=args.dataset_mixture.test_split_size, seed=seed
                )
                logger.info(
                    f"Split dataset into train and test sets with test size: {args.dataset_mixture.test_split_size}"
                )
                return combined_dataset
            else:
                return DatasetDict({"train": combined_dataset})
        else:
            raise ValueError("No datasets were loaded from the mixture configuration")

    else:
        raise ValueError("Either `dataset_name` or `dataset_mixture` must be provided")

In [ ]:
def get_tokenizer(model_args: ModelConfig, training_args: Any) -> PreTrainedTokenizer:
    """Get the tokenizer for the model."""
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
    )

    if training_args.chat_template is not None:
        tokenizer.chat_template = training_args.chat_template

    return tokenizer

In [ ]:
def get_model(model_args: ModelConfig, training_args: SFTConfig) -> AutoModelForCausalLM:
    """Get the model"""
    torch_dtype = (
        model_args.torch_dtype if model_args.torch_dtype in ["auto", None] else getattr(torch, model_args.torch_dtype)
    )
    quantization_config = get_quantization_config(model_args)
    model_kwargs = dict(
        revision=model_args.model_revision,
        trust_remote_code=model_args.trust_remote_code,
        attn_implementation=model_args.attn_implementation,
        torch_dtype=torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map=get_kbit_device_map() if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        **model_kwargs,
    )
    return model

In [ ]:
def get_callbacks(train_config, model_config) -> List[TrainerCallback]:
    callbacks = []
    # for callback_name in train_config.callbacks:
    #     if callback_name not in CALLBACKS:
    #         raise ValueError(f"Callback {callback_name} not found in CALLBACKS.")
    #     callbacks.append(CALLBACKS[callback_name](model_config))

    return callbacks

In [ ]:
def init_wandb_training(training_args):
    """
    Helper function for setting up Weights & Biases logging tools.
    """
    if training_args.wandb_entity is not None:
        os.environ["WANDB_ENTITY"] = training_args.wandb_entity
    if training_args.wandb_project is not None:
        os.environ["WANDB_PROJECT"] = training_args.wandb_project
    if training_args.wandb_run_group is not None:
        os.environ["WANDB_RUN_GROUP"] = training_args.wandb_run_group

### 訓練設定

In [ ]:
model_args = ModelConfig(
    # model_name_or_path="open-r1/Qwen2.5-Math-7B-RoPE-300k",
    model_name_or_path="Qwen/Qwen3-0.6B-Base",
    model_revision="main",
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
)

In [ ]:
training_args = SFTConfig(
    bf16=True,
    do_eval=False,
    eval_strategy="no",
    # gradient_accumulation_steps=8,
    gradient_accumulation_steps=32,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    hub_model_id="OpenR1-Distill-0.6B", # "OpenR1-Distill-7B"
    hub_strategy="every_save",
    learning_rate=4.0e-5,
    log_level="info",
    logging_steps=1,
    logging_strategy="steps",
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    packing=False,
    max_grad_norm=0.2,
    max_length=32768,
    max_steps=1, # -1
    num_train_epochs=1, # 5
    output_dir="data/OpenR1-Distill-0.6B", # "data/OpenR1-Distill-7B"
    overwrite_output_dir=True,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=2,
    push_to_hub=False, # True
    report_to=[], # ["wandb"]
    save_strategy="epoch",
    save_total_limit=1,
    seed=42,
    use_liger_kernel=True,
    warmup_ratio=0.03,
    dataset_num_proc=12,
    eos_token="<|im_end|>",
)

In [ ]:
# https://huggingface.co/datasets/open-r1/Mixture-of-Thoughts

script_args = ScriptArguments(
    dataset_name="open-r1/Mixture-of-Thoughts",
    dataset_config="math", # "all"
)

In [ ]:
set_seed(training_args.seed)

In [ ]:
# Check for last checkpoint
last_checkpoint = None

if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint is not None and training_args.resume_from_checkpoint is None:
    logger.info(f"Checkpoint detected, resuming training at {last_checkpoint=}.")

if "wandb" in training_args.report_to:
    init_wandb_training(training_args)

### データセットの読み込み

In [ ]:
dataset = get_dataset(script_args)
dataset

In [ ]:
# サブサンプリング
for split in dataset.keys():
    dataset[split] = dataset[split].select(range(1000))

dataset

### トークナイザーの読み込み

In [ ]:
tokenizer = get_tokenizer(model_args, training_args)
tokenizer

In [ ]:
if tokenizer.chat_template is None:
    logger.info("No chat template provided, defaulting to ChatML.")
    model, tokenizer = setup_chat_format(model, tokenizer, format="chatml")

tokenizer.chat_template

### モデルの読み込み

In [ ]:
model = get_model(model_args, training_args)
model

### SFTTrainerの作成

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset[script_args.dataset_train_split],
    eval_dataset=(dataset[script_args.dataset_test_split] if training_args.eval_strategy != "no" else None),
    processing_class=tokenizer,
    peft_config=get_peft_config(model_args),
    callbacks=get_callbacks(training_args, model_args),
)

### 訓練

In [ ]:
checkpoint = None

if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint

checkpoint

In [ ]:
train_result = trainer.train(resume_from_checkpoint=checkpoint)

### 保存

In [ ]:
metrics = train_result.metrics
metrics["train_samples"] = len(dataset[script_args.dataset_train_split])
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

In [ ]:
trainer.save_state()

### メモリ開放

In [ ]:
for target in ["dataset", "tokenizer", "model", "trainer", "trainer_result", "mertics"]:
    try:
        del globals()[target]
    except:
        pass

import gc
gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

## GRPO

```sh
!RANK=0 WORLD_SIZE=1 LOCAL_RANK=0 MASTER_ADDR=localhost MASTER_PORT=12345 python src/open_r1/grpo.py \
    --config recipes/DeepSeek-R1-Distill-Qwen-1.5B/grpo/config_demo.yaml \
    --model_name_or_path Qwen/Qwen3-0.6B-Base \
    --hub_model_id OpenR1-Distill-0.6B \
    --output_dir data/OpenR1-Distill-0.6B \
    --push_to_hub False \
    --report_to none \
    --vllm_mode colocate \
    --per_device_train_batch_size 1 \
    --num_generations 4 \
    --gradient_accumulation_steps 8
```

In [ ]:
from dataclasses import dataclass, field
from datasets import DatasetDict, concatenate_datasets
from datasets import load_dataset
from functools import partial, update_wrapper
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse, verify
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer
from transformers import set_seed
from transformers import TrainerCallback
from transformers.trainer_utils import get_last_checkpoint
from transformers.training_args import TrainingArguments
from trl import GRPOTrainer, ModelConfig, TrlParser, get_peft_config
from trl import ModelConfig, get_kbit_device_map, get_quantization_config
from trl import ModelConfig, SFTTrainer, TrlParser, get_peft_config, setup_chat_format
from typing import Any, Callable, Dict, Literal, List, Optional
import asyncio
import datasets
import json
import math
import os
import re
import subprocess
import sys
import transformers
import trl

logger.info(f"transformers: {transformers.__version__}") # 4.52.3
logger.info(f"torch: {torch.__version__}") # 2.6.0+cu124
logger.info(f"trl: {trl.__version__}") # 0.18.0

### 設定クラス

#### GRPOConfig

In [ ]:
@dataclass
class GRPOConfig(trl.GRPOConfig):
    """
    args for callbacks, benchmarks etc
    """

    benchmarks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "The benchmarks to run after training."},
    )
    callbacks: list[str] = field(
        default_factory=lambda: [],
        metadata={"help": "The callbacks to run during training."},
    )
    chat_template: Optional[str] = field(default=None, metadata={"help": "The chat template to use."})
    hub_model_revision: Optional[str] = field(
        default="main", metadata={"help": "The Hub model branch to push the model to."}
    )
    num_completions_to_print: int = field(default=0, metadata={"help": "Number of completions to print."})
    overwrite_hub_revision: bool = field(default=False, metadata={"help": "Whether to overwrite the Hub revision."})
    push_to_hub_revision: bool = field(default=False, metadata={"help": "Whether to push to a Hub revision/branch."})
    system_prompt: Optional[str] = field(
        default=None,
        metadata={"help": "The optional system prompt to use."},
    )
    wandb_log_unique_prompts: bool = field(
        default=True,
        metadata={
            "help": ("Whether to log the unique prompts to wandb. This will create a new run for each unique prompt.")
        },
    )
    wandb_entity: Optional[str] = field(
        default=None,
        metadata={"help": ("The entity to store runs under.")},
    )
    wandb_project: Optional[str] = field(
        default=None,
        metadata={"help": ("The project to store runs under.")},
    )
    wandb_run_group: Optional[str] = field(
        default=None,
        metadata={"help": ("The group to store runs under.")},
    )


#### GRPOScriptArguments

In [ ]:

@dataclass
class GRPOScriptArguments(ScriptArguments):
    """
    Script arguments for the GRPO training script.

    Args:
        reward_funcs (`list[str]`):
            List of reward functions. Possible values: 'accuracy', 'format', 'reasoning_steps', 'cosine', 'repetition_penalty', 'length', 'tag_count', 'code', 'ioi_code', 'code_format', 'soft_overlong_punishment'.
        cosine_min_value_wrong (`float`):
            Minimum reward for cosine scaling for wrong answers.
        cosine_max_value_wrong (`float`):
            Maximum reward for cosine scaling for wrong answers.
        cosine_min_value_correct (`float`):
            Minimum reward for cosine scaling for correct answers.
        cosine_max_value_correct (`float`):
            Maximum reward for cosine scaling for correct answers.
        cosine_max_len (`int`):
            Maximum length for cosine scaling.
        code_language (`str`):
            Language for code format reward.
        max_completion_len (`int`):
            Maximum number of tokens in completion.
        soft_punish_cache (`int`):
            Minimum number of tokens in completion.
    """

    reward_funcs: list[str] = field(
        default_factory=lambda: ["accuracy", "format", "tag_count"],
        metadata={
            "help": "List of reward functions. Possible values: 'accuracy', 'format', 'reasoning_steps', 'cosine', 'repetition_penalty', 'length', tag_count', 'code', 'code_format'"
        },
    )
    cosine_min_value_wrong: float = field(
        default=0.0,
        metadata={"help": "Minimum reward for wrong answers"},
    )
    cosine_max_value_wrong: float = field(
        default=-0.5,
        metadata={"help": "Maximum reward for wrong answers"},
    )
    cosine_min_value_correct: float = field(
        default=0.5,
        metadata={"help": "Minimum reward for correct answers"},
    )
    cosine_max_value_correct: float = field(
        default=1.0,
        metadata={"help": "Maximum reward for correct answers"},
    )
    cosine_max_len: int = field(
        default=1000,
        metadata={"help": "Maximum length for scaling"},
    )
    repetition_n_grams: int = field(
        default=3,
        metadata={"help": "Number of n-grams for repetition penalty reward"},
    )
    repetition_max_penalty: float = field(
        default=-1.0,
        metadata={"help": "Maximum (negative) penalty for for repetition penalty reward"},
    )
    code_language: str = field(
        default="python",
        # '(?:python|cpp)'
        metadata={
            "help": "Language for code format reward. Based on E2B supported languages https://e2b.dev/docs/code-interpreting/supported-languages",
            "choices": ["python", "javascript", "r", "java", "bash", "cpp"],
        },
    )
    code_eval_test_batch_size: int = field(
        default=1,
        metadata={
            "help": "for each generation, evaluate these many test cases in parallel, then check if any of them failed (0 score): if so stop evaluating; otherwise continue with the next batch of test cases. Useful to avoid overloading the eval server + save time on wrong solutions"
        },
    )
    code_eval_scoring_mode: Literal["pass_fail", "partial", "weighted_sum"] = field(
        default="weighted_sum",
        metadata={"help": "use fraction of passed test cases as reward. If false, use 0/1 scoring."},
    )
    parallel_code_exec_per_proc: int = field(
        default=2,
        metadata={
            "help": "Number of parallel E2B code executions per process. Default of 2 is suitable for the Free Hobby tier of E2B with 8 GPUs used for training."
        },
    )

    dataset_prompt_column: str = field(
        default="prompt",
        metadata={"help": "Column to use as prompts for training."},
    )

    e2b_router_url: Optional[str] = field(
        default=None,
        metadata={"help": "URL for the E2B router. See scripts/e2b_router.py"},
    )

    morph_router_url: Optional[str] = field(
        default=None,
        metadata={"help": "URL for the MorphCloud router. See scripts/morph_router.py"},
    )

    code_provider: Optional[str] = field(
        default="e2b",
        metadata={
            "help": "Provider for code execution. Options: 'e2b', 'local', 'morph'.",
            "choices": ["e2b", "local", "morph"],
        },
    )

    ioi_provider: Optional[str] = field(
        default="piston",
        metadata={
            "help": "Provider for IOI code execution. Options: 'piston', 'morph'.",
            "choices": ["piston", "morph"],
        },
    )

    max_completion_len: int = field(
        default=16384,
        metadata={"help": "Maximum number of characters in completion."},
    )
    soft_punish_cache: int = field(
        default=4096,
        metadata={"help": "Minimum number of characters in completion."},
    )

### 報酬関数

In [ ]:
def accuracy_reward(completions: list[list[dict[str, str]]], solution: list[str], **kwargs) -> list[Optional[float]]:
    """Reward function that checks if the completion is the same as the ground truth."""
    contents = [completion[0]["content"] for completion in completions]
    rewards = []
    for content, sol in zip(contents, solution):
        gold_parsed = parse(
            sol,
            extraction_mode="first_match",
        )
        if len(gold_parsed) != 0:
            # We require the answer to be provided in correct latex (no malformed operators)
            answer_parsed = parse(
                content,
                extraction_config=[
                    LatexExtractionConfig(
                        normalization_config=NormalizationConfig(
                            nits=False,
                            malformed_operators=False,
                            basic_latex=True,
                            equations=True,
                            boxed="all",
                            units=True,
                        ),
                        # Ensures that boxed is tried first
                        boxed_match_priority=0,
                        try_extract_without_anchor=False,
                    )
                ],
                extraction_mode="first_match",
            )
            # Compute binary rewards if verifiable, `None` otherwise to skip this example
            try:
                reward = float(verify(gold_parsed, answer_parsed))
            except Exception as e:
                print(f"verify failed: {e}, answer: {answer_parsed}, gold: {gold_parsed}")
                reward = None
        else:
            # If the gold solution is not parseable, we assign `None` to skip this example
            reward = None
            print("Failed to parse gold solution: ", sol)
        rewards.append(reward)

    return rewards

In [ ]:
def format_reward(completions, **kwargs):
    """Reward function that checks if the reasoning process is enclosed within <think> and </think> tags, while the final answer is enclosed within <answer> and </answer> tags."""
    pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content, re.DOTALL | re.MULTILINE) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]


In [ ]:
def tag_count_reward(completions, **kwargs) -> list[float]:
    """Reward function that checks if we produce the desired number of think and answer tags associated with `format_reward()`.

    Adapted from: https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb#file-grpo_demo-py-L90
    """

    def count_tags(text: str) -> float:
        count = 0.0
        if text.count("<think>\n") == 1:
            count += 0.25
        if text.count("\n</think>\n") == 1:
            count += 0.25
        if text.count("\n<answer>\n") == 1:
            count += 0.25
        if text.count("\n</answer>") == 1:
            count += 0.25
        return count

    contents = [completion[0]["content"] for completion in completions]
    return [count_tags(c) for c in contents]

In [ ]:
def get_reward_funcs(script_args) -> list[Callable]:
    REWARD_FUNCS_REGISTRY = {
        "accuracy": accuracy_reward,
        "format": format_reward,
        # "reasoning_steps": reasoning_steps_reward,
        # "cosine": get_cosine_scaled_reward(
        #     min_value_wrong=script_args.cosine_min_value_wrong,
        #     max_value_wrong=script_args.cosine_max_value_wrong,
        #     min_value_correct=script_args.cosine_min_value_correct,
        #     max_value_correct=script_args.cosine_max_value_correct,
        #     max_len=script_args.cosine_max_len,
        # ),
        # "repetition_penalty": get_repetition_penalty_reward(
        #     ngram_size=script_args.repetition_n_grams,
        #     max_penalty=script_args.repetition_max_penalty,
        # ),
        # "length": len_reward,
        # "code": update_wrapper(
        #     partial(
        #         code_reward,
        #         num_parallel=script_args.parallel_code_exec_per_proc,
        #         provider_type=script_args.code_provider,
        #         enforce_same_language=getattr(script_args, "enforce_same_language", False),
        #     ),
        #     code_reward,
        # ),
        # "binary_code": update_wrapper(
        #     partial(
        #         binary_code_reward,
        #         num_parallel=script_args.parallel_code_exec_per_proc,
        #         provider_type=script_args.code_provider,
        #         enforce_same_language=getattr(script_args, "enforce_same_language", False),
        #     ),
        #     binary_code_reward,
        # ),
        # "ioi_code": update_wrapper(
        #     partial(
        #         ioi_code_reward,
        #         test_batch_size=script_args.code_eval_test_batch_size,
        #         provider_type=getattr(script_args, "ioi_provider", "piston"),
        #     ),
        #     ioi_code_reward,
        # ),
        # "cf_code": update_wrapper(
        #     partial(
        #         cf_code_reward,
        #         test_batch_size=script_args.code_eval_test_batch_size,
        #         scoring_mode=script_args.code_eval_scoring_mode,
        #     ),
        #     cf_code_reward,
        # ),
        # "code_format": get_code_format_reward(language=script_args.code_language),
        "tag_count": tag_count_reward,
        # "soft_overlong_punishment": get_soft_overlong_punishment(
        #     max_completion_len=script_args.max_completion_len,
        #     soft_punish_cache=script_args.soft_punish_cache,
        # ),
    }
    reward_funcs = [REWARD_FUNCS_REGISTRY[func] for func in script_args.reward_funcs]

    return reward_funcs


### 訓練設定

In [ ]:
script_args = GRPOScriptArguments(
    dataset_name="open-r1/OpenR1-Math-220k",
    dataset_prompt_column="problem",
    reward_funcs=["accuracy", "format", "tag_count"],
)

In [ ]:
training_args = GRPOConfig(
    bf16=True,
    use_vllm=True,
    do_eval=False,
    # gradient_accumulation_steps=4,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # hub_model_id="DeepSeek-R1-Distill-Qwen-1.5B-GRPO",
    hub_model_id="DeepSeek-R1-Distill-Qwen-0.6B-GRPO",
    hub_strategy="every_save",
    learning_rate=1.0e-6,
    log_completions=True,
    log_level="info",
    logging_first_step=True,
    logging_steps=1,
    logging_strategy="steps",
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    max_prompt_length=512,
    max_completion_length=2048,
    max_steps=1, #-1,
    # num_generations=16,
    num_generations=4,
    num_train_epochs=1,
    # output_dir="data/DeepSeek-R1-Distill-Qwen-1.5B-GRPO",
    output_dir="data/DeepSeek-R1-Distill-Qwen-0.6B-GRPO",
    overwrite_output_dir=True,
    # per_device_eval_batch_size=16,
    per_device_eval_batch_size=1,
    # per_device_train_batch_size=16,
    per_device_train_batch_size=1,
    # push_to_hub=True,
    push_to_hub=False,
    # report_to=["wandb"],
    report_to=[],
    # reward_funcs=["accuracy", "format", "tag_count"],
    reward_weights=[1.0, 1.0, 1.0],
    save_strategy="epoch",
    save_total_limit=1,
    seed=42,
    temperature=0.7,
    use_liger_kernel=True,
    warmup_ratio=0.1,
    system_prompt="You are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer>",
    # chat_template="",
    vllm_mode="colocate",
)

In [ ]:
# model_name_or_path: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
# model_revision: main
# torch_dtype: bfloat16
# attn_implementation: flash_attention_2

model_args = ModelConfig(
    # model_name_or_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    model_name_or_path="Qwen/Qwen3-0.6B-Base",
    model_revision="main",
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
)

In [ ]:
set_seed(training_args.seed)

In [ ]:
# Check for last checkpoint
last_checkpoint = None

if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint is not None and training_args.resume_from_checkpoint is None:
    logger.info(f"Checkpoint detected, resuming training at {last_checkpoint=}.")

In [ ]:
if "wandb" in training_args.report_to:
    init_wandb_training(training_args)

### データセットの読み込み

In [ ]:
# Load the dataset
dataset = get_dataset(script_args)
dataset

In [ ]:
# デバッグ用に1000サンプルに制限

for split in dataset.keys():
    dataset[split] = dataset[split].select(range(1000))

dataset

In [ ]:
# Format into conversation
def make_conversation(example, prompt_column: str = script_args.dataset_prompt_column):
    prompt = []

    if training_args.system_prompt is not None:
        prompt.append({"role": "system", "content": training_args.system_prompt})

    if prompt_column not in example:
        raise ValueError(f"Dataset Question Field Error: {prompt_column} is not supported.")

    prompt.append({"role": "user", "content": example[prompt_column]})
    return {"prompt": prompt}

dataset = dataset.map(make_conversation)
dataset

In [ ]:
for split in dataset:
    if "messages" in dataset[split].column_names:
        dataset[split] = dataset[split].remove_columns("messages")

dataset

### トークナイザーの読み込み

In [ ]:
# トークナイザーを読み込み

tokenizer = get_tokenizer(model_args, training_args)
tokenizer

### モデルの読み込み

In [ ]:
# SFT済みモデルを読み込み

model = get_model(model_args, training_args)
model

### 報酬関数の読み込み

In [ ]:
# Get reward functions from the registry
reward_funcs = get_reward_funcs(script_args)
reward_funcs

### GRPOTrainerの作成

In [ ]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_funcs,
    args=training_args,
    train_dataset=dataset[script_args.dataset_train_split],
    eval_dataset=(dataset[script_args.dataset_test_split] if training_args.eval_strategy != "no" else None),
    peft_config=get_peft_config(model_args),
    callbacks=get_callbacks(training_args, model_args),
    processing_class=tokenizer,
)

In [ ]:
checkpoint = None

if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint

elif last_checkpoint is not None:
    checkpoint = last_checkpoint

### 訓練

In [ ]:
train_result = trainer.train(resume_from_checkpoint=checkpoint)

### 保存

In [ ]:
metrics = train_result.metrics
metrics["train_samples"] = len(dataset[script_args.dataset_train_split])
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
##################################
# Save model and create model card
##################################
# Align the model's generation config with the tokenizer's eos token
# to avoid unbounded generation in the transformers `pipeline()` function
trainer.model.generation_config.eos_token_id = tokenizer.eos_token_id
trainer.save_model(training_args.output_dir)

# Save everything else on main process
kwargs = {
    "dataset_name": script_args.dataset_name,
    "tags": ["open-r1"],
}
if trainer.accelerator.is_main_process:
    trainer.create_model_card(**kwargs)
    # Restore k,v cache for fast inference
    trainer.model.config.use_cache = True
    trainer.model.config.save_pretrained(training_args.output_dir)

In [ ]:
##########
# Evaluate
##########
if training_args.do_eval:
    metrics = trainer.evaluate()
    metrics["eval_samples"] = len(dataset[script_args.dataset_test_split])
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

In [ ]:
#############
# push to hub
#############
if training_args.push_to_hub:
    logger.info("Pushing to hub...")
    trainer.push_to_hub(**kwargs)